In [38]:
# Credit: https://www.depends-on-the-definition.com/classify-toxic-comments-on-wikipedia/

In [39]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.models import Model, Input
from keras.layers import Dense, Embedding, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam


plt.style.use("ggplot")

In [40]:
train_df = pd.read_csv("input/train.csv")

In [41]:
train_df.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [42]:
train_df.isna().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [43]:
## Replace all NaN values by "sterby". I don't know what that word means
# train_df = pd.read_csv("input/train.csv").fillna("sterby")
# test_df = pd.read_csv("input/test.csv").fillna("sterby")

train_df = pd.read_csv("input/train.csv")
test_df = pd.read_csv("input/test.csv")

In [44]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [45]:
X_train = train_df["comment_text"].values
y_train = train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test_df["comment_text"].values

In [46]:
max_features = 20000  # number of words we want to keep
maxlen = 100  # max length of the comments in the model
batch_size = 64  # batch size for the model
embedding_dims = 20  # dimension of the hidden variable, i.e. the embedding dimension

In [47]:
# Not sure what length of comments means. Is it the number of words or characters?
# What is embedding dimension

In [49]:
# Text data must be encoded as numbers to be used as input or output for machine learning and deep learning models.

In [55]:
tok = Tokenizer(num_words=max_features)
# tok.fit_on_texts(list(X_train) + list(X_test))
tok.fit_on_texts(X_train)
x_train = tok.texts_to_sequences(X_train)
x_test = tok.texts_to_sequences(X_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

159571 train sequences
153164 test sequences
Average train sequence length: 65
Average test sequence length: 57


In [60]:
# Can we use fit_on_texts on the testing data. Are we not tweaking the results there?

In [61]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (159571, 100)
x_test shape: (153164, 100)


In [62]:
input     = Input((maxlen,))
embedding = Embedding(max_features, embedding_dims, input_length=maxlen, 
                        embeddings_initializer="uniform")(input)
hidden    = GlobalMaxPooling1D()(embedding)
output    = Dense(6, activation='sigmoid')(hidden)
model = Model(inputs=input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])

In [63]:
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=3, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 9s 63us/step - loss: 0.0885 - accuracy: 0.9727 - val_loss: 0.0618 - val_accuracy: 0.9790
Epoch 2/3
143613/143613 [==============================] - 8s 52us/step - loss: 0.0538 - accuracy: 0.9809 - val_loss: 0.0583 - val_accuracy: 0.9804
Epoch 3/3
143613/143613 [==============================] - 10s 72us/step - loss: 0.0489 - accuracy: 0.9821 - val_loss: 0.0556 - val_accuracy: 0.9807
